<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebook'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# os.symlink('/content/drive/Shared drives/BigDATA TEAM 10', nb_path)
# sys.path.insert(0,nb_path)

In [ ]:
# !pip install --target=$nb_path flask-admin
# !apt-get install --target=$nb_path poppler-utils 
# !pip install --target=$nb_path pdf2image
# !pip install --target=$nb_path flask-socketio

In [42]:
!apt-get install --target=$nb_path poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [12]:
%cd /content/drive/Shared drives/BigDATA TEAM 10

/content/drive/Shared drives/BigDATA TEAM 10


In [ ]:
!git clone https://github.com/yoonkim313/dataCampusProject-Team10.git

In [15]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from flask_admin.contrib.fileadmin import FileAdmin
from flask_admin import Admin
from flask_dropzone import Dropzone
from pdf2image import convert_from_path
import logging

In [14]:
%cd /content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask

/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask


In [35]:
!pwd

/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask


In [11]:
!python /content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask/app.py

Traceback (most recent call last):
  File "/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask/app.py", line 2, in <module>
    from flask_admin.contrib.fileadmin import FileAdmin
ModuleNotFoundError: No module named 'flask_admin'


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

basedir = '/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask' # 현재 파일의 dir 절대경로, python console 에 쓰면 동일 결과
upload_dir = os.path.join(basedir, 'uploads')  # basedir 의 uploads 에 파일 저장
#################################################################################################################

admin = Admin(name='Uploaded Files')
admin.init_app(app)  # 이제 실행하고 주소창에 /admin 하면 창 나옴
dropzone = Dropzone(app)
admin.add_view(FileAdmin(upload_dir, name='FILES'))  # /admin 가면 올린 파일 관리 가능
app.config['DROPZONE_ALLOWED_FILE_CUSTOM'] = True
app.config['DROPZONE_ALLOWED_FILE_TYPE'] = 'image/*, .pdf, .txt'
#################################################################################################################


@app.route("/", methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        f = request.files.get('file')
        f.save(os.path.join(upload_dir, f.filename))
        images = convert_from_path(os.path.join(upload_dir, f.filename))
        for i, image in enumerate(images):
            fname = "uploads/image" + str(i) + ".jpg"
            image.save(fname, "JPEG")
    return render_template('homepage.html')
##############################################################################


@app.route("/result", methods=['GET', 'POST'])
def upload2():
    if request.method == 'POST':
        f = request.files.get('file')
        f.save(os.path.join(upload_dir, f.filename))
    return render_template('result.html')


# def main():
#     txt = ocr(text.jpeg)
#     html=highlight.main(txt)
#     processed = process(html)
#     return processed
# f.save(os.path.join(upload_dir, f.filename))

###############################################################################
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4295a85415f6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Aug/2020 16:21:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/css/mdb.min.css HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/js/popper.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/js/jquery-3.4.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:01] "GET /static/js/mdb.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:02] "GET /static/img/svg/arrow_left.svg HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:02] "GET /static/img/svg/arrow_right.svg HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:02] "GET /static/font/roboto/Roboto-Light.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2020 16:21:02] "GET /static/font/roboto/Roboto-Reg

In [ ]:
%tb

SystemExit: ignored

In [ ]:
! python3 /content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/ocr/deep-text-recognition/train.py

In [ ]:
!python /content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/main.ipynb